<a href="https://colab.research.google.com/github/Kaanxx/Alpaca2ShareGPT/blob/main/ALPACA_ShareGPT_d%C3%B6n%C3%BC%C5%9Ft%C3%BCr%C3%BCc%C3%BC_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ALPACA JSONL (Instruct) Dataset Formatını ShareGPT (Chat) Formatına dönüştürücü.


In [ ]:
import json
from google.colab import files
import os
import io

print("Lütfen dönüştürmek istediğiniz Alpaca formatındaki JSONL (.jsonl) dosyasını seçin:")

# Kullanıcıdan dosya yüklemesini isterr
uploaded = files.upload()

# Yükleme başarılı olmadıysa veya dosya seçilmediyse çık
if not uploaded:
    print("\nDosya yüklenmedi. İşlem iptal edildi.")
else:
    # Yüklenen dosyanın adını al
    input_alpaca_filename = list(uploaded.keys())[0]
    print(f"\n'{input_alpaca_filename}' dosyası yüklendi.")

    # Yüklenen dosyanın içeriğini al (byte olarak)
    alpaca_content_bytes = uploaded[input_alpaca_filename]

    # Çıktı ShareGPT JSONL dosyasının adını belirle
    output_sharegpt_filename = os.path.splitext(input_alpaca_filename)[0] + "_sharegpt.jsonl"
    print(f"Çıktı dosyası '{output_sharegpt_filename}' olarak kaydedilecek.")

    processed_lines = 0
    skipped_lines = 0
    error_occurred = False

    print("\nDönüştürme işlemi başlıyor...")

    try:
        # Byte içeriğini metin olarak işlemek için StringIO kullan
        # Dosyanın UTF-8 ile kodlandığını varsayıyoruz, değilse değiştirin
        alpaca_content_stream = io.StringIO(alpaca_content_bytes.decode('utf-8'))

        # Çıktı JSONL dosyasını yazma modunda aç (UTF-8 encoding)
        with open(output_sharegpt_filename, 'w', encoding='utf-8') as outfile:
            # Alpaca dosyasını satır satır oku
            for line_num, line in enumerate(alpaca_content_stream):
                line = line.strip() # Başındaki/sonundaki boşlukları temizle
                if not line: # Boş satırları atla
                    continue

                try:
                    # Her satırı JSON nesnesine dönüştür
                    alpaca_data = json.loads(line)

                    # Gerekli alanların varlığını kontrol et
                    instruction = alpaca_data.get("instruction")
                    output = alpaca_data.get("output")

                    if not instruction or not output:
                        print(f"Uyarı: Satır {line_num + 1} 'instruction' veya 'output' alanı eksik olduğu için atlanıyor.")
                        skipped_lines += 1
                        continue

                    # Opsiyonel 'input' alanını al
                    input_text = alpaca_data.get("input", "") # Yoksa boş string döndür

                    # 'human' değerini oluştur: instruction + (varsa) input
                    human_value = instruction.strip() # Önce instruction'ı al
                    if input_text and input_text.strip(): # Eğer input varsa ve boş değilse
                        # Instruction ile input arasına genellikle çift satır boşluk konur
                        human_value += "\n\n" + input_text.strip()

                    # 'gpt' değerini oluştur
                    gpt_value = output.strip()

                    # ShareGPT formatındaki girdiyi oluştur
                    sharegpt_entry = [
                        {"from": "human", "value": human_value},
                        {"from": "gpt", "value": gpt_value}
                    ]

                    # ShareGPT girdisini JSON string'ine çevir (non-ASCII karakterleri koru)
                    json_string = json.dumps(sharegpt_entry, ensure_ascii=False)

                    # JSON string'ini çıktı dosyasına yaz ve yeni satır ekle
                    outfile.write(json_string + '\n')
                    processed_lines += 1

                except json.JSONDecodeError:
                    print(f"Hata: Satır {line_num + 1} geçerli bir JSON değil, atlanıyor: {line[:100]}...") # Hatalı satırın başını göster
                    skipped_lines += 1
                except Exception as e:
                    print(f"Hata: Satır {line_num + 1} işlenirken beklenmedik bir hata oluştu: {e}")
                    skipped_lines += 1


        print(f"\nDönüştürme tamamlandı!")
        print(f"Toplam {processed_lines} satır başarıyla işlendi.")
        if skipped_lines > 0:
            print(f"Toplam {skipped_lines} satır atlandı (eksik alan veya format hatası nedeniyle).")
        print(f"Sonuçlar '{output_sharegpt_filename}' dosyasına yazıldı.")

    except UnicodeDecodeError:
        print("\nHata: Dosya UTF-8 olarak okunamadı. Dosya kodlaması farklı olabilir.")
        error_occurred = True
    except Exception as e:
        print(f"\nBeklenmedik bir genel hata oluştu: {e}")
        error_occurred = True
        # Hata oluşursa yarım kalan dosyayı sil
        if os.path.exists(output_sharegpt_filename):
            os.remove(output_sharegpt_filename)
            print(f"Hata nedeniyle '{output_sharegpt_filename}' dosyası silindi.")

    # Hata oluşmadıysa ve en az bir satır işlendiyse dosyayı indir
    if not error_occurred and processed_lines > 0:
        print(f"\n'{output_sharegpt_filename}' dosyası indiriliyor...")
        files.download(output_sharegpt_filename)
    elif not error_occurred and processed_lines == 0:
         print(f"\n'{output_sharegpt_filename}' dosyası oluşturuldu ancak işlenecek geçerli satır bulunamadı.")
         # İsterseniz boş dosyayı da indirebilirsiniz:
         # files.download(output_sharegpt_filename)
    elif error_occurred:
        print("\nDosya indirme işlemi hata nedeniyle yapılamadı.")

Lütfen dönüştürmek istediğiniz Alpaca formatındaki JSONL (.jsonl) dosyasını seçin:


Saving 300 YENİ 4_cleaned.jsonl to 300 YENİ 4_cleaned.jsonl

'300 YENİ 4_cleaned.jsonl' dosyası yüklendi.
Çıktı dosyası '300 YENİ 4_cleaned_sharegpt.jsonl' olarak kaydedilecek.

Dönüştürme işlemi başlıyor...

Dönüştürme tamamlandı!
Toplam 570 satır başarıyla işlendi.
Sonuçlar '300 YENİ 4_cleaned_sharegpt.jsonl' dosyasına yazıldı.

'300 YENİ 4_cleaned_sharegpt.jsonl' dosyası indiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>